In [21]:
!ls

Linear Regression - Project Exercise - Solutions.ipynb
Linear Regression with Python.ipynb
Position_Salaries.csv
USA_Housing.csv
Untitled.ipynb
__init__.py
bitcoin_price_1week_test.csv
bitcoin_price_1week_train.csv
bitcoin_test.csv
bitcoin_train.csv
celery_app
celerybeat-schedule
celerybeat.pid
utils


In [22]:
import numpy as np
import pandas as pd

In [23]:
# Importing the dataset
tr_data = pd.read_csv('bitcoin_train.csv')
tr_data.head()

,Open,High,Low,Close
0,2763.24,2889.62,2720.61,2875.34
1,2724.39,2758.53,2644.85,2757.18
2,2807.02,2808.76,2692.80,2726.45
3,2679.73,2897.45,2679.73,2809.01
4,2538.71,2693.32,2529.34,2671.78


In [24]:
ts_data = pd.read_csv('bitcoin_test.csv')
ts_data.head()

,Open,High,Low,Close
0,327.16,331.77,325.08,330.49
1,325.57,334.00,325.48,327.55
2,326.07,329.05,320.63,325.89
3,338.65,340.53,321.06,325.75
4,345.01,348.05,337.14,338.32


In [26]:
ts_data.shape

(556, 4)

In [28]:
def preprocess_dataframe(df, out_col):

    df["close_1"] = df[out_col].shift(-1)
    df["close_10"] = df[out_col].shift(-10)

    df.drop(columns=[out_col], axis=1, inplace=True)
    df.dropna(inplace=True)

In [31]:
preprocess_dataframe(tr_data, "Close")
preprocess_dataframe(ts_data, "Close")

In [33]:
ts_data.head()

,Open,High,Low,close_1,close_10
0,327.16,331.77,325.08,327.55,347.27
1,325.57,334.00,325.48,325.89,358.35
2,326.07,329.05,320.63,325.75,358.42
3,338.65,340.53,321.06,338.32,383.16
4,345.01,348.05,337.14,345.31,386.48


In [35]:
Xtrain = tr_data[['Open', 'High', 'Low']]
Ytrain = tr_data[["close_1", "close_10"]]
Xtest = ts_data[['Open', 'High', 'Low']]
Ytest = ts_data[["close_1", "close_10"]]

In [44]:
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

gbr = GradientBoostingRegressor()
# gbr = BaggingRegressor()
# gbr = RandomForestRegressor()
model = MultiOutputRegressor(estimator=gbr)
print(model)

MultiOutputRegressor(estimator=GradientBoostingRegressor())


In [45]:
from time import time
start = time()
model.fit(Xtrain, Ytrain)
end = time()

score = model.score(Xtrain, Ytrain)
print("Training score: {} total time: {}".format(score, end - start))

Training score: 0.9965228583349892 total time: 0.23864483833312988


In [48]:
from sklearn.metrics import mean_squared_error
Ypred = model.predict(Xtest)
print("y MSE:%.4f" % mean_squared_error(Ytest.iloc[:,1], Ypred[:,1]))

y MSE:17330.0821


In [38]:
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import metrics

In [49]:
out_test = np.array(Ytest.iloc[:,1])
out_model = Ypred[:,1]
res_data = {}

test_score = r2_score(out_test, out_model)
spearman = spearmanr(out_test, out_model)
pearson = pearsonr(out_test, out_model)
print(f'Test data R-2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')

res_data["r2_score"] = test_score
res_data["spearman_correlation"] = spearman[0]
res_data["pearson_correlation"] = pearson[0]

MAE_VAL = metrics.mean_absolute_error(out_test, out_model)
MSE_VAL = metrics.mean_squared_error(out_test, out_model)
RMSE_VAL = np.sqrt(metrics.mean_squared_error(out_test, out_model))

res_data["MeanAbsoluteError"] = MAE_VAL
res_data["MeanSquaredError"] = MSE_VAL
res_data["RootMeanSquaredError"] = RMSE_VAL

print('Mean Absolute Error:', MAE_VAL)
print('Mean Squared Error:', MSE_VAL)
print('Root Mean Squared Error:', RMSE_VAL)

Test data R-2 score: 0.744
Test data Spearman correlation: 0.916
Test data Pearson correlation: 0.887
Mean Absolute Error: 105.18183765253083
Mean Squared Error: 17330.082128723545
Root Mean Squared Error: 131.64376980595605


In [91]:
from joblib import dump, load
# dump(model, 'mimo_gbr.joblib')

In [92]:
loaded_mdl = load('mimo_gbr.joblib')

In [60]:
print(list(Xtest.iloc[0, :]))
print(list(Ytest.iloc[0, :]))

[327.16, 331.77, 325.08]
[327.55, 347.27]


In [94]:
result = loaded_mdl.predict([[327.16, 331.77, 325.08]])

In [97]:
result[0][1]

337.44994435100983

## checking generator

In [99]:
import csv

def read_csv_data(path, columns):
    with open(path) as file:
        reader = csv.DictReader(file)
        for row in reader:
            data = dict(row)
            yield {k.lower(): float(data[k]) for k in columns}

In [100]:
data_cols = ["Open","High","Low","Close"]
data_path = "bitcoin_test.csv"
gen_d = read_csv_data(data_path, data_cols)

In [103]:
next(gen_d, None)

{'open': 326.07, 'high': 329.05, 'low': 320.63, 'close': 325.89}

In [54]:
gen_d

<generator object read_csv_data at 0x7f1467b06750>